<h1 class="font-effect-3d" style="font-family:Sancreek; color:#39d4be;">Code Modules, Styling, and Links</h1>

#### [GitHub Repository](https://github.com/OlgaBelitskaya/deep_learning_projects/tree/master/DL_PP2) & [Colaboratory Version](https://colab.research.google.com/drive/1Z9Fz0OOi6bpWvH-H2OhExC9CkGPWBYZz)

In [ ]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Sancreek|Roboto&effect=3d');
span {font-family:Roboto; color:black; text-shadow:4px 4px 4px #aaa;}  
a,h4,div.output_area pre{font-family:Roboto; font-size:120%; color:#39d4be;}      
</style>

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import numpy as np,pandas as pd 
import os,h5py,pylab as pl
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from scipy import misc
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Conv2D,MaxPooling2D,GlobalAveragePooling2D
from keras.applications.nasnet import NASNetLarge,NASNetMobile
from keras.applications.nasnet import preprocess_input as nnpi
print(os.listdir("../input"))

<h1 class="font-effect-3d" style="font-family:Sancreek; color:#39d4be;">Preprocess the Data</h1>

In [ ]:
fpath="../input/classification-of-handwritten-letters/"
letters=u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
f=h5py.File(fpath+'LetterColorImages_123.h5','r')
keys=list(f.keys()); n=2700
backgrounds=np.array(f[keys[0]])[:n]
images=np.array(f[keys[1]])[:n]/255
labels=np.array(f[keys[2]])[:n]
[keys,
 [backgrounds.shape,images.shape,labels.shape]]

In [ ]:
ri=np.random.randint(n)
print('Label: ', letters[labels[ri]-1])
pl.figure(figsize=(3,3))
pl.imshow(images[ri]); pl.show()

In [ ]:
print(set(labels))
print("{а, б, в, г, д ,е, ё, ж, з, и, й, к, л, м, н, о, п, "+\
      "р, с, т, у, ф, х, ц, ч, ш, щ, ъ, ы, ь, э, ю, я}")

In [ ]:
# one-hot encoding the labels, started from the zero label
cat_labels=to_categorical(np.array(labels-1),33)
cat_labels.shape

In [ ]:
# splitting the data
x_train,x_test,y_train,y_test=\
train_test_split(images,cat_labels,test_size=.2,random_state=1)
m=int(len(x_test)/2)
x_valid,y_valid=x_test[:m],y_test[:m]
x_test,y_test=x_test[m:],y_test[m:]
print([x_train.shape,x_valid.shape,x_test.shape,
       y_train.shape,y_valid.shape,y_test.shape])

In [ ]:
del images,labels,cat_labels,backgrounds

<h1 class="font-effect-3d" style="font-family:Sancreek; color:#39d4be;">Classification Models</h1>
#### NASnetLarge

In [ ]:
# creating bottleneck features
x_train=np.array([misc.imresize(x_train[i],(331,331,3)) 
                  for i in range(0,len(x_train))]).astype('float32')
x_valid=np.array([misc.imresize(x_valid[i],(331,331,3)) 
                  for i in range(0,len(x_valid))]).astype('float32')
x_test=np.array([misc.imresize(x_test[i],(331,331,3)) 
                 for i in range(0,len(x_test))]).astype('float32')
x_train=nnpi(x_train)
x_valid=nnpi(x_valid)
x_test=nnpi(x_test)
fn="../input/keras-applications-weights/nasnet_large_no_top.h5"
nasnet_base_model=NASNetLarge(weights=fn,include_top=False)
x_train=nasnet_base_model.predict(x_train)
x_valid=nasnet_base_model.predict(x_valid)
x_test=nasnet_base_model.predict(x_test)

In [ ]:
def nasnet_model():
    model = Sequential()   
    model.add(GlobalAveragePooling2D(input_shape=x_train.shape[1:]))    
    model.add(Dense(2048))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.5))        
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.5))    
    model.add(Dense(33,activation='softmax'))     
    model.compile(loss='categorical_crossentropy',
                  optimizer='nadam',metrics=['accuracy'])
    return model
nasnet_model=nasnet_model()

In [ ]:
fw='weights.best.nasnet.hdf5'
checkpointer=ModelCheckpoint(filepath=fw,verbose=2,save_best_only=True)
lr_reduction=ReduceLROnPlateau(monitor='val_loss',patience=5,
                               verbose=2,factor=.5)
estopping=EarlyStopping(monitor='val_loss',patience=25,verbose=2)
history=nasnet_model.fit(x_train,y_train,
                         validation_data=(x_valid,y_valid),
                         epochs=100,batch_size=128,verbose=2,
                         callbacks=[checkpointer,lr_reduction,estopping]);

In [ ]:
# loading the model with the best validation accuracy
nasnet_model.load_weights(fw)
# calculations of classification accuracy on the testing set
scores=nasnet_model.evaluate(x_test,y_test)
print("Accuracy: %.2f%%"%(scores[1]*100))
scores

In [ ]:
del x_train,x_valid,x_test,y_train,y_valid,y_test

#### NASNetMobile

In [ ]:
f=h5py.File(fpath+'LetterColorImages_123.h5','r')
keys=list(f.keys()); n=4000
images=np.array(f[keys[1]])[:n]/255
labels=np.array(f[keys[2]])[:n]
cat_labels=to_categorical(np.array(labels-1),33)
x_train,x_test,y_train,y_test=\
train_test_split(images,cat_labels,test_size=.2,random_state=1)
m=int(len(x_test)/2)
x_valid,y_valid=x_test[:m],y_test[:m]
x_test,y_test=x_test[m:],y_test[m:]
print([x_train.shape,x_valid.shape,x_test.shape,
       y_train.shape,y_valid.shape,y_test.shape])
del images,labels,cat_labels

In [ ]:
# creating bottleneck features
x_train=np.array([misc.imresize(x_train[i],(224,224,3)) \
                  for i in range(0,len(x_train))]).astype('float32')
x_valid=np.array([misc.imresize(x_valid[i],(224,224,3)) \
                  for i in range(0,len(x_valid))]).astype('float32')
x_test=np.array([misc.imresize(x_test[i],(224,224,3)) \
                 for i in range(0,len(x_test))]).astype('float32')
x_train=nnpi(x_train)
x_valid=nnpi(x_valid)
x_test=nnpi(x_test)
fn="../input/keras-applications-weights/nasnet_mobile_no_top.h5"
nasnet_base_model=NASNetMobile(weights=fn,include_top=False)
x_train=nasnet_base_model.predict(x_train)
x_valid=nasnet_base_model.predict(x_valid)
x_test=nasnet_base_model.predict(x_test)

In [ ]:
def nasnet_model():
    model=Sequential()   
    model.add(GlobalAveragePooling2D(input_shape=x_train.shape[1:]))    
    model.add(Dense(2048))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.5))        
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.5))    
    model.add(Dense(33,activation='softmax'))    
    model.compile(loss='categorical_crossentropy',
                  optimizer='nadam',metrics=['accuracy'])
    return model
nasnet_model=nasnet_model()

In [ ]:
fw='weights.best.nasnet.hdf5'
checkpointer=ModelCheckpoint(filepath=fw,verbose=2,save_best_only=True)
lr_reduction=ReduceLROnPlateau(monitor='val_loss',patience=5,
                               verbose=2,factor=.5)
estopping=EarlyStopping(monitor='val_loss',patience=25,verbose=2)
history=nasnet_model.fit(x_train,y_train,
                         validation_data=(x_valid,y_valid),
                         epochs=100,batch_size=128,verbose=2,
                         callbacks=[checkpointer,lr_reduction,estopping]);

In [ ]:
# loading the model with the best validation accuracy
nasnet_model.load_weights(fw)
# calculations of classification accuracy on the testing set
scores=nasnet_model.evaluate(x_test,y_test)
print("Accuracy: %.2f%%"%(scores[1]*100))
scores

In [ ]:
del x_train,x_valid,x_test,y_train,y_valid,y_test